In [1]:
import sys

# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../..')

import tqdm
import my_utils
import os
import numpy as np
import time
from datetime import datetime        
import shutil
import cv2
import csv
from utils_1 import *
import glob

from alpha_blending_tool import *


np.random.seed(123456)
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
def initialize_dataset(dataset, feature_dir, label_dir, new):
    if new :
        if os.path.exists(dataset):
            shutil.rmtree( dataset )
        os.mkdir( dataset )
        os.mkdir( feature_dir )
        os.mkdir( label_dir )

In [3]:
# create_dataset(training_csv, training, training_feature_dir, training_label_dir, prediction_csv, augment_color)
def create_dataset(wound_paths, dataset, feature_dir, label_dir, prediction_csv=None, augment_args=[], num=None, new=True, withMorphological=False ):
    
    initialize_dataset(dataset, feature_dir, label_dir, new)
    count       = 1
    bg_only     = []
    
    for wound_path in wound_paths[:]:
        imageDirPath = wound_path
        image_name  = getDirName(wound_path.split("/")[-1])
        _, _, files = next( os.walk(wound_path) )
        print("wound_path", wound_path)
        # get path images
        original_image_path     = os.path.join( wound_path, image_name + '.JPG' )
        granulation_image_paths = [ os.path.join( wound_path, p ) for p in files if 'Granulation' in p and not '.tif' in p ]
        slough_image_paths      = [ os.path.join( wound_path, p ) for p in files if 'Slough' in p and not '.tif' in p ]
        necrosis_image_paths    = [ os.path.join( wound_path, p ) for p in files if 'Necrosis' in p and not '.tif' in p ]
        epithelial_image_paths  = [ os.path.join( wound_path, p ) for p in files if 'Epithelial' in p and not '.tif' in p ]
        
        if not os.path.exists(original_image_path):
            print("file not exist", original_image_path)
            continue
        else:
            pass
        # read images
        original_image     = my_utils.read_image_rgb(original_image_path)
        granulation_images = [ my_utils.read_image_grayscale(p) for p in granulation_image_paths ]
        slough_images      = [ my_utils.read_image_grayscale(p) for p in slough_image_paths ]
        necrosis_images    = [ my_utils.read_image_grayscale(p) for p in necrosis_image_paths ]
        epithelial_images  = [ my_utils.read_image_grayscale(p) for p in epithelial_image_paths ]
        
      
        
        height, width, _   = original_image.shape
       
        # create bg for mask 
        granulation_image = np.full( fill_value=0, shape=(height, width), dtype=np.uint8 )
        slough_image      = np.full( fill_value=0, shape=(height, width), dtype=np.uint8 )
        necrosis_image    = np.full( fill_value=0, shape=(height, width), dtype=np.uint8 )
        epithelial_image  = np.full( fill_value=0, shape=(height, width), dtype=np.uint8 )
        
        for img in granulation_images:
            img = (img == 255) * 255
            granulation_image = (( granulation_image + img ) > 127 ) * 255

        for img in slough_images:
            img = (img == 255) * 255
            slough_image = (( slough_image + img ) > 127 ) * 255

        for img in necrosis_images:
            img = (img == 255) * 255
            necrosis_image = (( necrosis_image + img ) > 127 ) * 255

        for img in epithelial_images:
            img = (img == 255) * 255
            epithelial_image = (( epithelial_image + img ) > 127 ) * 255
    
        epithelial_image  = ( epithelial_image == 255 ) * 4
        slough_image      = ( slough_image == 255 ) * 3
        granulation_image = ( granulation_image == 255 ) * 2
        necrosis_image    = ( necrosis_image == 255 ) * 1
        epithelial_image_unique, epithelial_image_counts = np.unique(epithelial_image, return_counts=True)
        
        bg   = np.full(fill_value = 0, shape = (height, width), dtype = np.uint8) * 0
        temp = np.stack([bg, necrosis_image, granulation_image, slough_image, epithelial_image], axis = -1)
        temp = np.argmax(temp, axis = -1)
        label_image = temp * color

        if np.unique(label_image) == np.ndarray([0]):
            bg_only.append(image_name + x1)
     
        def plot(original_image, epithelial_image, slough_image, granulation_image, necrosis_image):
            f, axs = plt.subplots(1,6, figsize=(16,16))
            
            epithelial_image  = epithelial_image * color
            slough_image      = slough_image * color
            granulation_image = granulation_image * color
            necrosis_image    = necrosis_image * color
            
            axs[0].imshow( original_image, vmin=0, vmax=255, cmap='gray' )
            axs[1].imshow( granulation_image, vmin=0, vmax=255, cmap='gray' )
            axs[2].imshow( slough_image, vmin=0, vmax=255, cmap='gray' )
            axs[3].imshow( necrosis_image, vmin=0, vmax=255, cmap='gray' )
            axs[4].imshow( epithelial_image, vmin=0, vmax=255, cmap='gray' )
            axs[5].imshow( label_image, vmin=0, vmax=255, cmap='gray' )
            plt.show()
            
#         original_images = None
        if num != None and count == num+1:
            break
        else:
            plot(original_image, epithelial_image, slough_image, granulation_image, necrosis_image)
#             name = ( "%d" % count ).zfill(5) + ".png"
            name = imageDirPath.split(os.sep)[-1]
#             print(imageDirPath)
#             print(name)
            new_label_dir = os.path.join(label_dir, name)
#             print(new_label_dir)
            
            createDir(new_label_dir)
            label_image_path = os.path.join( new_label_dir, "old-{}.png".format(name) if withMorphological == True else "{}.png".format(name) )
            if new:
                my_utils.write_image_grayscale( path=label_image_path, image=label_image)
            
#             if withMorphological == True:
                # Dilation followed by Erosion. 
#                 kernel = np.ones((7,7), np.uint8)
#                 granulation_image = cv2.morphologyEx(np.array(granulation_image, dtype=np.uint8), cv2.MORPH_CLOSE, kernel)
#                 slough_image      = cv2.morphologyEx(np.array(slough_image, dtype=np.uint8), cv2.MORPH_CLOSE, kernel)
#                 necrosis_image    = cv2.morphologyEx(np.array(necrosis_image, dtype=np.uint8), cv2.MORPH_CLOSE, kernel)
#                 epithelial_image  = cv2.morphologyEx(np.array(epithelial_image, dtype=np.uint8), cv2.MORPH_CLOSE, kernel)

#                 bg   = np.full(fill_value = 0, shape = (height, width), dtype = np.uint8) * 0
#                 temp = np.stack([bg, necrosis_image, granulation_image, slough_image, epithelial_image], axis = -1)
#                 kernel = np.ones((3,3), np.uint8)
#                 temp = cv2.dilate(temp, kernel, iterations = 1)
#                 temp = np.argmax(temp, axis = -1)
#                 label_image = temp * color
#     #             print(imageDirPath)
#     #             print(name)
#                 new_label_dir = os.path.join(label_dir, name)
#     #             print(new_label_dir)

#                 createDir(new_label_dir)
#                 label_image_path = os.path.join( new_label_dir, name + ".png" )
#                 if new:
#                     my_utils.write_image_grayscale( path=label_image_path, image=label_image)
                
        count += 1

    print(bg_only)
# create_dataset(imageFullPat1List, training, training_feature_dir, training_label_dir, '', [])

In [4]:
def getImageInDir(imageDir):
    return glob.glob(testImageDir)

def mockUser(size=5):
    return ["uid_%s" % (str(uid).zfill(3)) for uid in range(1, size + 1)]

def createDir(path_dir):
    if not os.path.exists(path_dir):
        os.makedirs(path_dir)

In [5]:
def inverseBgColor(image1D):
    image1D[image1D > 0] = -1
    image1D[image1D == 0] = 1
    image1D[image1D == -1] = 0
    return image1D

def calculate_iou( y_true, y_pred ):
    y_pred = y_pred[...,1:].flatten()
    y_true = y_true[...,1:].flatten()
    
    intersection = np.sum( y_pred * y_true )
    union = np.sum( y_pred ) + np.sum( y_true ) - intersection
    print(intersection, union)
    if union == 0.:
        return 100.
    
    iou_score = intersection / union
    return iou_score * 100

def calculate_bg_iou( y_true, y_pred ):
    y_pred = y_pred[...,1:].flatten()
    y_true = y_true[...,1:].flatten()
    
    y_pred = inverseBgColor(y_pred)
    y_true = inverseBgColor(y_true)
    
    intersection = np.sum( y_pred * y_true )
    union = np.sum( y_pred ) + np.sum( y_true ) - intersection
    
    if union == 0.:
        return 100.
    iou_score = intersection / union
    return iou_score * 100


def calculate_dice( y_true, y_pred ):
    y_pred = y_pred[...,1:].flatten()
    y_true = y_true[...,1:].flatten()
    
    intersection = np.sum( y_pred * y_true )
    denom        = np.sum( y_pred ) + np.sum( y_true )
    
    if denom == 0.:
        return 100.
    
    dice_score = 2. * intersection / denom
    return dice_score * 100
    
def calculate_bg_dice( y_true, y_pred ):
    y_pred = y_pred[...,1:].flatten()
    y_true = y_true[...,1:].flatten()
    
    y_pred = inverseBgColor(y_pred)
    y_true = inverseBgColor(y_true)
    
    intersection = np.sum( y_pred * y_true )
    denom        = np.sum( y_pred ) + np.sum( y_true )
    
    if denom == 0.:
        return 100.
    
    dice_score = 2. * intersection / denom
    return dice_score * 100    
    
def calculate_region_size_error( y_true, y_pred ):
    y_pred = y_pred[...,1:].flatten()
    y_true = y_true[...,1:].flatten()
    
    gt = np.sum(y_true)
    cs = np.sum(y_pred)
    
    if gt == 0.:
        return 100.
    
    region_size_error = abs( gt - cs ) / gt
    return region_size_error * 100

In [6]:
def labelGray2Color(image, color_class):
    label_color = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    label_color[image == 0]    = color_class[0]
    label_color[image == color*1] = color_class[1]
    label_color[image == color*2] = color_class[2]
    label_color[image == color*3] = color_class[3]
    label_color[image == color*4] = color_class[4]
    return label_color

def get_y( path ):
    grayscale = read_image( path, 'gray' ) / color
    layers = [  (grayscale == i) + [0] for i in range(0, num_classes) ]   
    return np.stack( layers, axis=2 ).astype( np.float32 )

def get_image( path ):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    return img

def resize_image(image, size):
    return cv2.resize(image, size, interpolation=cv2.INTER_NEAREST)

def compareDiffImage( image1, image2):
    label_color = np.zeros((image1.shape[0], image1.shape[1], 3), dtype=np.uint8)
    label_color[image1 > 0] = 80
    label_color[image1 != image2] = 255
    return label_color

def compareDiffImageNoTissue( image1, image2):
    label_color = np.zeros((image1.shape[0], image1.shape[1], 3), dtype=np.uint8)
    label_color[image1 > 0] = 80
    label_color[image1 != image2] = 255
    return label_color
    
def plot_sample(title, images):
    imageSize = len(images)
    print(imageSize)
    f, axs = plt.subplots( 1, imageSize  + 1, figsize=(24,15) )
    f.suptitle( title )
    axs = axs.ravel()
    
    for index, item in enumerate(images):
        axs[index].set_title(item["label"])
        axs[index].imshow(item["image"], cmap="gray")
#     a1.set_title('original')
#     a1.imshow(original)
    
#     a2.set_title('label1')
#     a2.imshow( label1)
    
#     a3.set_title('Label')
#     a3.imshow(label2)
    
#     a4.set_title('pred vs label IoU')
#     a4.imshow( compareImage )
    
#     a4.set_title('pred vs label IoU')
#     a4.imshow( mergeLabelPred )
    
#     a4.set_title('pred vs label IoU')
#     a4.imshow( mergeLabelPred )
    
#     a4.set_title('pred vs label IoU')
#     a4.imshow( mergeLabelPred )
    
    axs[imageSize].imshow(colorClassImage)
    return f

def cv2WriteImage(path, image, isColor = False):
    try:
        splitPath = path.split(os.sep)
        newImageName = "color_{}".format(splitPath[-1]) if isColor == True else splitPath[-1]
        newPath = os.path.join(os.sep.join(splitPath[:-1]), newImageName)
        cv2.imwrite(newPath, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
    except Exception as e:
        print(e)
        
def createColorClassImage(color_class):
    class_height = 300
    height = len(color_class) * class_height 
    width = height // len(color_class)
    img = np.zeros((height, width, 3), dtype= np.uint8)
    print(img.shape)
    for i, color in enumerate(color_class):
        if i+1>len(color_class):
            break
        y1 = i*class_height
        y2 = (i+1)*class_height
        img[y1: y2, :] = color
        text = tmp_labels[i]

        # font 
        font = cv2.FONT_HERSHEY_SIMPLEX 

        # org 
        org = (10, y2-(class_height//2)) 

        # fontScale 
        fontScale = 1.5

        # Red color in BGR 
        color = (255, 255, 255) if i == 0 else (0, 0, 0) 

        # Line thickness of 3 px 
        thickness = 3

        # Using cv2.putText() method 
        image = cv2.putText(img, text, org, font, fontScale,  
                     color, thickness, cv2.LINE_AA, False) 
       
    return img

In [7]:
def getNoTissueImage(image):
    tmp = image.copy().argmax(axis=-1 )
    
    c, uni = np.unique(tmp, return_counts=True)
    kernel = np.ones((7,7), np.uint8)
    tmp     = cv2.morphologyEx(np.array(tmp, dtype=np.uint8), cv2.MORPH_CLOSE, kernel)
    
    kernel = np.ones((5,5), np.uint8)
    tmp = cv2.dilate(tmp, kernel, iterations = 1)
    c, uni = np.unique(tmp, return_counts=True)

    tmp[tmp>0]= 1
    kernel = np.ones((5,5), np.uint8)
  
    return tmp

class CompareResult:
    def __init__(self, image1, image2, user1, user2, imageID, noTissue1, noTissue2):
        self.user1   = user1
        self.user2   = user2
        self.imageId = imageId
        
        self.noTissueIoU = calculate_iou(noTissue1, noTissue2)
        self.noTissueDice = calculate_dice(noTissue1, noTissue2)
        
        self.iou     = calculate_iou(image1, image2)
        self.bg_iou  = calculate_bg_iou(image1, image2)
        self.dice    = calculate_dice(image1, image2)
        self.bg_dice = calculate_bg_dice(image1, image2)
        
    def get_iou(self):
        return np.round(self.iou, 2)
    
    def get_bg_iou(self):
        return np.round(self.bg_iou, 2)
    
    def get_dice(self):
        return np.round(self.dice, 2)
    
    def get_bg_dice(self):
        return np.round(self.bg_dice, 2)
    
    def get_no_tissue_iou(self):
        return np.round(self.noTissueIoU, 2)
        
    def get_no_tissue_dice(self):
        return np.round(self.noTissueDice, 2)
    

    
class CompareImage:
    CompareList = {}
    UserList = []
    ImageList = []
    SummaryLogArray = []
    ProcessCount = 0
    __SummaryLog = ""
    __SummaryLogCSV = ""
    
    def __init__(self, verbose=True, transpose=False, overWrite=True):
        self.Verbose   = verbose
        self.Transpose = transpose
        self.overWrite = overWrite
        self.re_color_class = np.reshape(color_class, (5, 1, 3))
        print('compare config:\nverbose:{}\ntranspose: {}\n'.format(verbose, transpose) )
        
    def compare(self, imagePath1, imagePath2, user1, user2, imageID):
        self.ProcessCount += 1
#         print("round: {}\timageID:{: <20}".format(self.ProcessCount, imageID))
     
        user1 = str(user1+1)
        user2 = str(user2+1)
    
        imageID = imagePath1.split(os.sep)[-1]
        
        image1 = get_y(imagePath1)
        image2 = get_y(imagePath2)
        
        noTissue1 = getNoTissueImage(image1)
        noTissue2 = getNoTissueImage(image2)
        nt1, ntc1 = np.unique(noTissue1,  return_counts=True)
        nt2, ntc2 = np.unique(noTissue2,  return_counts=True)
        
#         print("nt 1", nt1, ntc1)
#         print("nt 2", nt2, ntc2)
        
            
        userCoupleKey = self._makeCoupleUserKey(user1, user2)
        compareResult = CompareResult(image1, image2, user1, user2, imageID, noTissue1, noTissue2)
        self.updateCompareList(userCoupleKey, imageId, compareResult)
        self.updateUserList(user1)
        self.updateUserList(user2)
        self.updateImageList(imageId)
        
        self.__genSummaryArray()
        self.__summary()
        self.__summaryCSV()
        
        figHeader = "imageId:{: <15} \nuser: {}, {}\n".format(imageId, user1, user2)
        figHeader += "image1:{}\n".format(imagePath1.split(os.sep)[-1])
        figHeader += "image2:{}\n".format(imagePath2.split(os.sep)[-1])
        figHeader += "IoU:{}\n".format(compareResult.get_iou())
        figHeader += "Dice:{}\n".format(compareResult.get_dice())
        figHeader += "non_bg_iou:{}\n".format(compareResult.get_no_tissue_iou())
        figHeader += "non_bg_dice:{}\n".format(compareResult.get_no_tissue_dice())
        
#         figHeader += "bg IoU:{}\n".format(compareResult.get_bg_iou())
        
        originalPath  = os.path.join( path, imageId,'{}.JPG'.format(imageId))
        originalImage = cv2.cvtColor(cv2.imread(originalPath), cv2.COLOR_BGR2RGB)
        
        image1 = get_image(imagePath1)
        image2 = get_image(imagePath2)
      
        
        image1_withColor = labelGray2Color(image1, color_class)
        image2_withColor = labelGray2Color(image2, color_class)
        
        compareImage = compareDiffImage( image1.copy(), image2.copy())
        compareNoTissueImage = compareDiffImageNoTissue( noTissue1.copy(), noTissue2.copy())
        
        alpha_blending_no_tissue = self.getAlphaBlendingImage(originalImage, noTissue1, noTissue2, imageId, user1, user2)
        
        
        self.getAlphaBlendingImageExtractTissue(originalImage, image1, image2, imageId, user1, user2)
#         plotImages = (
#             {"label":"originalImage",
#              "image":originalImage},
#             {"label": "image1", 
#              "image":image1_withColor},
#             {"label":"image2",
#              "image":image2_withColor},
#             {"label":"compare",
#              "image": compareImage},
#             {"label":"compare No Tissue",
#              "image": compareNoTissueImage}
#         )
         #don't fill tissue color
#         plotImages = (
#             {"label":"originalImage",
#              "image":originalImage},
#             {"label": "image1", 
#              "image":image1},
#             {"label":"image2",
#              "image":image2},
#             {"label":"compare",
#              "image": compareImage},
#         )
        
        # fill tissue color
        plotImages = (
            {"label":"originalImage",
             "image":originalImage},
            {"label": "image1", 
             "image":image1_withColor},
            {"label":"image2",
             "image":image2_withColor},
#             {"label":"alpha_blending_no_tissue",
#              "image": alpha_blending_no_tissue},
             {"label":"compare",
             "image": compareImage},
        )
        
        f_ = plot_sample(figHeader, plotImages)
                    
        if self.Verbose:
            plt.show()
            
        figure_path = os.path.join(export_figure_dir, imageId)
        createDir(figure_path)
        f_.savefig(os.path.join(figure_path, "{}.png".format(userCoupleKey)))
        
        # copy originalimage to result
        shutil.copy2(originalPath, os.path.join(figure_path, '{}.JPG'.format(imageId)))

#         # copy label1 and label 2 to result if not exists
        self.writeLabelImage(figure_path, user1, imagePath1, image1_withColor)
        self.writeLabelImage(figure_path, user2, imagePath2, image2_withColor)
        
        
        self.copySrcImageToResult(figure_path, user1, imagePath1, image1)
        self.copySrcImageToResult(figure_path, user2, imagePath2, image2)
        
        newPathCompareImage = '{}_compare_{}_{}.png'.format(imageId, user1, user2)
        newPathCompareImage = os.path.join(figure_path, newPathCompareImage)
        if not os.path.exists(newPathCompareImage) or self.overWrite:
            cv2WriteImage(newPathCompareImage, compareImage)
        
        newPathCompareImageNoTissue = '{}_compare_no_tissue_{}_{}.png'.format(imageId, user1, user2)
        newPathCompareImageNoTissue = os.path.join(figure_path, newPathCompareImageNoTissue)
        if not os.path.exists(newPathCompareImageNoTissue) or self.overWrite:
            cv2WriteImage(newPathCompareImageNoTissue, compareNoTissueImage)
        
      
        
        self.showSummaryLog()
    
    def getAlphaBlendingImageExtractTissue(self, originalImage, image1, image2, imageId, user1, user2):
        cUnique1 = countUnique(image1)
        cUnique2 = countUnique(image2)
               
#         print("cUnique1",cUnique1)
#         print("cUnique2",cUnique2)
        alphaBlendingImages = []
        
        alpha_blending_image_dir = os.path.join(export_figure_dir, imageId , "alpha_blending_images")
        createDir(alpha_blending_image_dir)
        
        for i, classLabel in enumerate(tmp_labels):
            if i == 0:
                continue
                
            color_focus = color * i
            tmpTissue1 = np.zeros_like(image1)
            tmpTissue1[image1 == color_focus] = 190
            
            tmpTissue2 = np.zeros_like(image2)
            tmpTissue2[image2 == color_focus] = 190
            
            tmpTissue1 = cv2.cvtColor(tmpTissue1, cv2.COLOR_GRAY2RGB)
            tmpTissue2 = cv2.cvtColor(tmpTissue2, cv2.COLOR_GRAY2RGB)
            
            alphaBlendingImage = blend_adv(originalImage, tmpTissue1, tmpTissue2)
            alphaBlendingImages += [alphaBlendingImage]
            
            newPathAlphaBlendingImage = '{}_{}_{}_{}.png'.format(imageId, classLabel, user1, user2)
            newPathAlphaBlendingImage = os.path.join(alpha_blending_image_dir, newPathAlphaBlendingImage)
            if not os.path.exists(newPathAlphaBlendingImage) or self.overWrite:
                cv2WriteImage(newPathAlphaBlendingImage, alphaBlendingImage)

        return alphaBlendingImages
     
        
        
    
    def getAlphaBlendingImage(self, originalImage, image1, image2, imageId, user1, user2):
        imageGray1 = image1.copy()
        imageGray1 *= 190
        
        imageGray2 = image2.copy()
        imageGray2 *= 190
        
        imageGray1 = cv2.cvtColor(imageGray1, cv2.COLOR_GRAY2RGB)
        imageGray2 = cv2.cvtColor(imageGray2, cv2.COLOR_GRAY2RGB)
        
        alphaBlendingImage = blend_adv(originalImage, imageGray1, imageGray2)
        
        alpha_blending_image_dir = os.path.join(export_figure_dir, imageId , "alpha_blending_images")
        createDir(alpha_blending_image_dir)
        
        newPathAlphaBlendingImage = '{}_non_bg_{}_{}.png'.format(imageId, user1, user2)
        newPathAlphaBlendingImage = os.path.join(alpha_blending_image_dir, newPathAlphaBlendingImage)
        if not os.path.exists(newPathAlphaBlendingImage) or self.overWrite:
            cv2WriteImage(newPathAlphaBlendingImage, alphaBlendingImage)
            
        return alphaBlendingImage
        
    
    def writeLabelImage(self, figure_path, uid, imagePath, image):
        newPath = "user_{}_{}.png".format(uid, imagePath.split('/')[-1].replace('.png','').replace('.jpg','').replace('.JPG',''), uid)
        newPath = os.path.join(figure_path, newPath)
        if not os.path.exists(newPath) or self.overWrite:
#             print("not exits", newPathLabel1)
            cv2WriteImage(newPath, image, isColor=True)
#             shutil.copy2(imagePath, os.path.join(figure_path, newPath))
        
    def copySrcImageToResult(self, figure_path, uid, imagePath, image):
        newPath = "user_{}_{}.png".format(uid, imagePath.split('/')[-1].replace('.png','').replace('.jpg','').replace('.JPG',''))
        newPath = os.path.join(figure_path, newPath)
        if not os.path.exists(newPath) or self.overWrite:
#             print("not exits", newPathLabel1)
#             cv2WriteImage(newPath, image, isColor=True)
            shutil.copy2(imagePath, os.path.join(figure_path, newPath))   
        
    def updateCompareList(self, userCoupleKey, imageId, CompareResult):
        try:
            self.CompareList[userCoupleKey][imageId] = CompareResult
        except:
            self.CompareList[userCoupleKey] = {}
            self.CompareList[userCoupleKey][imageId] = CompareResult
    
    def updateUserList(self, user):
        if user not in self.UserList:
            self.UserList.append(str(user))
            
    def updateImageList(self, imageId):
        if imageId not in self.ImageList:
            self.ImageList.append(str(imageId))
    
    def _makeCoupleUserKey(self, user1, user2):
        return "{} - {}".format(user1, user2)
    
    def __genSummaryLogFromArray(self, file_format="txt"):
        
        summaryLogArray = self.SummaryLogArray
        
        summmary_sep = "%15s " if file_format == "txt" else "%15s,"
        summary_template = summmary_sep * (len(summaryLogArray[0])) + "\n"
        summaryLog = summary_template % ( *summaryLogArray[0],)
        
        for item in summaryLogArray[1:]:
            summaryLog += summary_template % ( *item[:],)

        if file_format == "txt": self.__SummaryLog = summaryLog  
        if file_format == "csv": self.__SummaryLogCSV = summaryLog
            
        summaryLogArray = self.SummaryLogArrayTranspose
        
        summmary_sep = "%15s " if file_format == "txt" else "%15s,"
        summary_template = summmary_sep * (len(summaryLogArray[0])) + "\n"
        summaryLog = summary_template % ( *summaryLogArray[0],)
        
        for item in summaryLogArray[1:]:
            summaryLog += summary_template % ( *item[:],)   
            
        if file_format == "txt": self.__SummaryLogTranspose = summaryLog  
        if file_format == "csv": self.__SummaryLogCSVTranspose = summaryLog
    
    def __getAverage(self, items):
        return np.round(np.sum(items)/len([x for x in items if x != 0]), 2)
    
    def __getStandardDeviation(self, items):
        return np.round(np.std([x for x in items if x != 0]), 2)
    
    def __extractData(self, compareItem):
        iouList = []
        imageList   = sorted(self.ImageList)
        for imageId in imageList:
            try:
                item = compareItem[imageId]
                iouList += [[item.get_iou(), item.get_bg_iou(), item.get_dice(), item.get_bg_dice(), item.get_no_tissue_iou(), item.get_no_tissue_dice()]]
            except Exception as e:
                iouList += [[0, 0, 0, 0, 0, 0]]
                print("errorinextract",e)
        result = np.transpose(iouList)
        iou_list, bg_iou_list, dice_list, bg_dice_list, no_tissue_iou_list, no_tissue_dice_list = result 
        return iou_list, bg_iou_list, dice_list, bg_dice_list, no_tissue_iou_list, no_tissue_dice_list

    def __genSummaryArray(self):
        avgLabelList = ["avg Iou", "std IoU", "avg dice", 'std dice', "avg non_bg_iou", 'std non_bg_iou', "avg non_bg_dice", 'std non_bg_dice']
        userList = sorted(self.UserList)
        userCoupleKeyList = [i for i in np.array(np.meshgrid(userList, userList)).T.reshape(-1,2) if i[0] != i[1]]
        
        compareList = self.CompareList
        imageList   = sorted(self.ImageList)
        summaryLogArray = [['imageName', *[",,,%s," % (imageName) for imageName in imageList], *avgLabelList]]
#         summaryLogArray += [['', *['iou | bg_iou | dice | bg_dice' for _ in range(len(imageList))], *['' for _ in range(4)]]]
        summaryLogArray += [["user", *['iou , dice, non_bg_iou, non_bg_dice,' for _ in range(len(imageList))], *['' for _ in range(len(avgLabelList))]]]
        
        
        for (user1, user2) in userCoupleKeyList:
            userKey = self._makeCoupleUserKey(user1, user2)
            try:
                compareItem = compareList[userKey]
                
                iou_list, bg_iou_list, dice_list, bg_dice_list, no_tissue_iou_list, no_tissue_dice_list = self.__extractData(compareItem)
                print(no_tissue_dice_list)
                mergeIoUDiceToString = ["{} , {}, {}, {},".format(iou, dice, no_tissue_iou, no_tissue_dice) for iou, bg_iou, dice, bg_dice, no_tissue_iou, no_tissue_dice in zip(iou_list, bg_iou_list, dice_list, bg_dice_list, no_tissue_iou_list, no_tissue_dice_list)] 
                summaryLogArray += [[userKey,\
                                     *mergeIoUDiceToString,\
                                     self.__getAverage(iou_list),\
                                     self.__getStandardDeviation(iou_list),\
                                     self.__getAverage(dice_list),\
                                     self.__getStandardDeviation(dice_list),\
                                     self.__getAverage(no_tissue_iou_list),\
                                     self.__getStandardDeviation(no_tissue_iou_list),\
                                     self.__getAverage(no_tissue_dice_list),\
                                     self.__getStandardDeviation(no_tissue_dice_list),\
                                    ]]
            except Exception as e:
                print("catch on genSummary", e)
                pass
            
        self.SummaryLogArray          = summaryLogArray
        self.SummaryLogArrayTranspose = list(map(list, zip(*summaryLogArray)))
    
    def __summary(self):
        self.__genSummaryLogFromArray(file_format="txt")
        
    def __summaryCSV(self):
        self.__genSummaryLogFromArray(file_format="csv")
        
    def showSummaryLog(self, transpose=False):
        if not (transpose or self.Transpose): print(self.__SummaryLog)   
        if transpose or self.Transpose: print(self.__SummaryLogTranspose)   
        print("#"*20 + " END ROUND: {} ".format(self.ProcessCount) + "#"*20 +"\n\n\n\n\n\n")
        
    def showSummaryLogCSV(self, transpose=False):
        if not (transpose or self.Transpose): print(self.__SummaryLogCSV)
        if (transpose or self.Transpose): print(self.__SummaryLogCSVTranspose)      
        print("#"*20 + " END ROUND: {} ".format(self.ProcessCount) + "#"*20 +"\n\n\n\n\n\n")
    
    def save_log_txt(self, details=[], transpose=False):
        file_name = "compare_log_{}.txt".format(datetime.now().strftime("%m-%d-%Y"))
        with open(os.path.join(export_summary_dir, file_name), "a+") as f:
            f.write("\n\n\n{}\n".format(str(datetime.now().strftime("%m-%d-%Y:%H:%M:%S"))))
            f.write( details + "\n\n")
            if not (transpose or self.Transpose): f.write(self.__SummaryLog)
            if (transpose or self.Transpose): f.write(self.__SummaryLogTranspose)  
        
    def save_log_csv(self, transpose=False):
        file_name = "compare_log_{}.csv".format(str(time.time()))
        with open(os.path.join(export_summary_dir, file_name), "w+") as f:
            if not (transpose or self.Transpose): f.write(self.__SummaryLogCSV)
            if (transpose or self.Transpose): f.write(self.__SummaryLogCSVTranspose) 
                

In [8]:
def getDirName(fullDirPath):
    dirName = fullDirPath.split(os.sep)[-1]
    splitDirName = dirName.split("_")
    resultDirName = splitDirName[0] + "_" + splitDirName[1]

    return resultDirName

def getNameFromDirs(path):
    allDirs = [getDirName(fullDirPath) for fullDirPath in glob.glob(os.path.join(path, '*'))]

    return sorted(list(set(allDirs)))

def getNameImageInList(path, nameImageList):
    resultNameImage = []
    for nameImage in nameImageList: 
        dirNames = []
        for fullDirPath in glob.glob(os.path.join(path, '*')):
            dirName = fullDirPath.split(os.sep)[-1]
            
            if nameImage in dirName:
                dirNames.append(dirName)
           
        resultNameImage.append(sorted(dirNames))
    return resultNameImage

### CONFIG

In [9]:
# resize config
width  = 512
height = 512

root        = os.getcwd()
color_class = [(0, 0, 0), (128, 0, 64), (230, 0, 0),  (255, 255, 102), (255, 153, 102)]
tmp_labels  = ['background', 'necrosis', 'granulation', 'slough', 'epithelial']
color         = 63
num_classes   = 5

path = 'data/multimask_rev2_split_file'
imageFullPat1List = glob.glob(os.path.join(path, '*'))
dirTarget = "compare_repair_common"
dirResultTarget = "_".join(["result", "color", dirTarget])
print(dirResultTarget)


# training        = os.path.join(root, "data", "compare")
# training        = os.path.join(root, "data", "compare_repair")
training        = os.path.join(root, "data", dirTarget)


training_feature_dir = os.path.join(training, "feature")
training_label_dir   = os.path.join(training, "label")
print("training_label_dir", training_label_dir)

# result_dir         = os.path.join(root, "compare_result")
# result_dir         = os.path.join(root, "compare_repair_result")
result_dir         = os.path.join(root, dirResultTarget)

export_summary_dir = os.path.join(result_dir, 'log')
export_figure_dir = os.path.join(result_dir, 'figure')

createDir(result_dir)
createDir(export_summary_dir)
createDir(export_figure_dir)
    
# imagePath1 = "data/testing/label/00001.png"
# imagePath2 = "data/testing/label/00002.png"
# imageId = 4
# user1 = "00001"
# user2 = "00002"
# testRoot = "data/testing/label"
# testImageDir = os.path.join(testRoot, "*.png")
# testImages = getImageInDir(testImageDir)
# testUsers = mockUser()
# testLoopNumber = 90
# print(len(testImages), testImages)
# print(len(testUsers), testUsers)

compareParams = {
    'verbose'   : 0,
    'transpose' : 0, # transpose summary table  
    'overWrite' : 1,
}
compareImage = CompareImage(**compareParams)


colorClassImage = createColorClassImage(color_class)
cv2WriteImage(os.path.join(export_figure_dir, "colorClass.png"),colorClassImage)

result_color_compare_repair_common
training_label_dir /home/minibear-l/Desktop/woundSee/data/compare_repair_common/label
compare config:
verbose:0
transpose: 0

(1500, 300, 3)


### Pre Process


In [10]:
# create_dataset(imageFullPat1List, training, training_feature_dir, training_label_dir, '', [])
# create_dataset(imageFullPat1List, training, training_feature_dir, training_label_dir, '', [], withMorphological=True)

### TEST COMPARE

In [11]:

nameImageList = getNameFromDirs(training_label_dir)
nameImages    = getNameImageInList(training_label_dir, nameImageList)

In [ ]:


def getImageDir(imagePath, file_type = "png"):
    imageDir = os.path.join(training_label_dir, imagePath, "{}.{}".format(imagePath,file_type))
    return glob.glob(imageDir)[0]

for index, nameImage in enumerate(nameImages[:]):    
    user1 = 0
    user2 = 1
    user3 = 2
    print("round: {}\timageID:{: <20}".format(index, nameImage[0]))
    
    image1 = getImageDir(imagePath=nameImage[user1])
    image2 = getImageDir(nameImage[user2])
    image3 = getImageDir(nameImage[user3])
    print(image1)
    print(image2)
    print(image3)
#     break
    imageId = getDirName(image1).split('.')[0]
    compareImage.compare(image1, image2, user1, user2, imageId)
    compareImage.compare(image1, image3, user1, user3, imageId)    
    compareImage.compare(image2, image3, user2, user3, imageId)    
compareImage.save_log_csv()
import time
print("process_time", time.process_time)


round: 0	imageID:0001_1              
/home/minibear-l/Desktop/woundSee/data/compare_repair_common/label/0001_1/0001_1.png
/home/minibear-l/Desktop/woundSee/data/compare_repair_common/label/0001_1_34/0001_1_34.png
/home/minibear-l/Desktop/woundSee/data/compare_repair_common/label/0001_1_67/0001_1_67.png
28340 32297
24676.0 31552.0
[93.47]
catch on genSummary '2 - 1'
4
      imageName      ,,,0001_1,         avg Iou         std IoU        avg dice        std dice  avg non_bg_iou  std non_bg_iou avg non_bg_dice std non_bg_dice 
           user iou , dice, non_bg_iou, non_bg_dice,                                                                                                                                 
          1 - 2 78.21 , 87.77, 87.75, 93.47,           78.21             0.0           87.77             0.0           87.75             0.0           93.47             0.0 

#################### END ROUND: 1 ####################






29875 33091
27483.0 31801.0
[93.47]
[94.89]
catch 

/home/minibear-l/.local/lib/python3.5/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


      imageName      ,,,0001_1,      ,,,0002_2,      ,,,0002_3,      ,,,0002_4,      ,,,0003_5,      ,,,0004_6,      ,,,0004_7,         avg Iou         std IoU        avg dice        std dice  avg non_bg_iou  std non_bg_iou avg non_bg_dice std non_bg_dice 
           user iou , dice, non_bg_iou, non_bg_dice, iou , dice, non_bg_iou, non_bg_dice, iou , dice, non_bg_iou, non_bg_dice, iou , dice, non_bg_iou, non_bg_dice, iou , dice, non_bg_iou, non_bg_dice, iou , dice, non_bg_iou, non_bg_dice, iou , dice, non_bg_iou, non_bg_dice,                                                                                                                                 
          1 - 2 78.21 , 87.77, 87.75, 93.47, 47.14 , 64.07, 86.55, 92.79, 34.43 , 51.22, 87.89, 93.55, 62.67 , 77.05, 93.73, 96.77, 73.99 , 85.05, 84.08, 91.35, 82.33 , 90.31, 91.78, 95.71, 79.0 , 88.27, 91.88, 95.77,            65.4           16.94           77.68            13.7           89.09            3.19            94.2          

In [ ]:
# compareImage.save_log_csv()


In [ ]:

# for i in range(testLoopNumber):
#     sampleImages = np.random.choice(testImages, 2, replace=False)
#     img1 = sampleImages[0]
#     img2 = sampleImages[1]
    
#     sampleUsers = np.random.choice(testUsers, 2, replace=False)
#     user1 = sampleUsers[0]
#     user2 = sampleUsers[1]
    
#     imageId = img1.split('/')[-1]
#     compareImage.compare(img1, img2, user1, user2, imageId)
#     log_detail = "\n".join([img1, img2])
# #     compareImage.showSummaryLog()
# #     compareImage.showSummaryLog(transpose=True)
# #     compareImage.showSummaryLogCSV()
# #     compareImage.showSummaryLogCSV(transpose=True)
# #     compareImage.save_log_txt(log_detail)
    
# # compareImage.showSummaryLog()
# # compareImage.showSummaryLog(transpose=True)
# # compareImage.showSummaryLogCSV()
# # compareImage.showSummaryLogCSV(transpose=True)
# # compareImage.save_log_continue(log_detail)
# compareImage.save_log_csv(transpose=True)


In [ ]:
# create_dataset(training_csv, training, training_feature_dir, training_label_dir, prediction_csv, augment_color)


# 